<a href="https://colab.research.google.com/github/jloutz/Resume_NER/blob/master/flair_nlp_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resume NER Part 4: Working with Flair NLP

---

In this part we will use flair NLP to train a model on our data and evaluate the results. Please make sure you have set up your Google account and uploaded your files to Google drive. This Notebook should run on Google Colab.

Let's change the working directory to the Google drive where our training data is, and install flair nlp. 

In [29]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir("/content/gdrive/My Drive/SAKI_2019/flair") 

In [31]:
# download flair library #
! pip install flair

In the next section, we will train a NER model with flair. This code is taken from the flair nlp tutorials section 7. "Training a model" 
https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md



In [32]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher

## make sure this describes your file structure
columns = {0: 'text', 1: 'ner'}

# folder where training and test data are
data_folder = '/content/gdrive/My Drive/SAKI_2019/flair'

# 1.0 is full data, try a much smaller number like 0.1 to test run the code
downsample = 1.0 

## your train file name
train_file = 'training_bilou_2.txt'

## your test file name
test_file = 'test_bilou_2.txt'
# 1. get the corpus
corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                             train_file=train_file,
                                                             test_file=test_file,
                                                           dev_file=None).downsample(downsample)
print(corpus)

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')
print(tag_dictionary.idx2item)


2019-06-18 20:05:34,843 Reading data from /content/gdrive/My Drive/SAKI_2019/flair
2019-06-18 20:05:34,852 Train: /content/gdrive/My Drive/SAKI_2019/flair/training_bilou_2.txt
2019-06-18 20:05:34,857 Dev: None
2019-06-18 20:05:34,861 Test: /content/gdrive/My Drive/SAKI_2019/flair/test_bilou_2.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


Corpus: 30597 train + 3400 dev + 11218 test sentences
[b'<unk>', b'O', b'B-Designation', b'I-Designation', b'L-Designation', b'B-Degree', b'I-Degree', b'L-Degree', b'"B-Companies', b'"I-Companies', b'"L-Companies', b'U-Degree', b'U-Designation', b'"U-Companies', b'<START>', b'<STOP>']


In [33]:

# 4. initialize embeddings. Experiment with different embedding types to see what gets the best results
from typing import List
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings,FlairEmbeddings, CharacterEmbeddings 

embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    # comment in this line to use character embeddings
    CharacterEmbeddings(),

    # comment in these lines to use flair embeddings (needs a LONG time to train :-)
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type='ner',
                                        use_crf=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

## give your model a name and folder of your choice. Your model will be saved there for loading later 
## you can run this notebook many times with different embeddings/params and save the models with different names
model_name = 'resources/taggers/resume-ner-word_char_double_flair_BL2_1_0'

# 7. start training - you can experiment with batch size if you get memory errors
# how many epochs does it take before the model stops showing improvement? Start with a big number like 150, and stop the code cell
# from running at any time - the framework will persist the best model even if you interrupt training. 
trainer.train(model_name,
              learning_rate=0.1,
              mini_batch_size=32,
              #anneal_with_restarts=True,
              checkpoint=True,
              max_epochs=30)




2019-06-18 20:06:55,812 ----------------------------------------------------------------------------------------------------
2019-06-18 20:06:55,820 Evaluation method: MICRO_F1_SCORE
2019-06-18 20:06:56,588 ----------------------------------------------------------------------------------------------------
2019-06-18 20:06:58,441 epoch 1 - iter 0/957 - loss 27.00418854
2019-06-18 20:07:42,942 epoch 1 - iter 95/957 - loss 2.85799920
2019-06-18 20:08:23,428 epoch 1 - iter 190/957 - loss 2.09480588
2019-06-18 20:09:03,357 epoch 1 - iter 285/957 - loss 1.68658280
2019-06-18 20:09:47,009 epoch 1 - iter 380/957 - loss 1.45715005
2019-06-18 20:10:26,256 epoch 1 - iter 475/957 - loss 1.31876280
2019-06-18 20:11:05,785 epoch 1 - iter 570/957 - loss 1.20127872
2019-06-18 20:11:45,640 epoch 1 - iter 665/957 - loss 1.10641432
2019-06-18 20:12:28,667 epoch 1 - iter 760/957 - loss 1.02856486
2019-06-18 20:13:08,757 epoch 1 - iter 855/957 - loss 0.98011125
2019-06-18 20:13:48,375 epoch 1 - iter 950/9